In [1]:
# pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [20]:
#library required
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy import text
import os
import time 
from datetime import datetime
import io 
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error


In [56]:
#ASIN Classification: Preparation

file_path = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B1 katchon\0_Data\katchon_Amazon_com_weekly_data.csv"

# Read the CSV file into a DataFrame
df_base = pd.read_csv(file_path)
df_base = df_base[df_base['Week_Start_date'] <= '2024-08-26']

# Display the DataFrame
df_base.head()

# print(len(df_base))
print("Asin count in base data: ", df_base.asin.nunique())
df_base_with_gmv_gt_zero = df_base[df_base['gmv_units']>0]
print("Asin count where gmv_units > 0: ", df_base_with_gmv_gt_zero.asin.nunique())


sales_week_df = df_base_with_gmv_gt_zero.groupby('asin').agg({ 'Week_Start_date':['nunique','min', 'max'],'gmv_units':['mean',np.std]}).reset_index()
sales_week_df.columns = [' '.join(col).strip() for col in sales_week_df.columns.values]
sales_week_df


sales_week_df['Week_Start_date max'] = pd.to_datetime(sales_week_df['Week_Start_date max'], format='%Y-%m-%d')
sales_week_df['Week_Start_date min'] = pd.to_datetime(sales_week_df['Week_Start_date min'], format='%Y-%m-%d')
sales_week_df['n_weeks'] = ((sales_week_df['Week_Start_date max'] - sales_week_df['Week_Start_date min']).dt.days)/7 
# sales_week_df
sales_week_df['ADI'] =  sales_week_df['n_weeks'] / (sales_week_df['Week_Start_date nunique'])

# eg: B0D7HZR4XL - Some edge case here ADI going > 1

sales_week_df['CV_2'] = (sales_week_df['gmv_units std']/sales_week_df['gmv_units mean'])**2

sales_week_df

def classification(x):
    if x['ADI']<1.32 and x['CV_2']<0.49:
        return 'Smooth'
    elif x['ADI']>=1.32 and x['CV_2']<0.49:
        return 'Intermittent'
    elif x['ADI']<1.32 and x['CV_2']>=0.49:
        return 'Erratic'
    elif x['ADI']>=1.32 and x['CV_2']>=0.49:
        return 'Lumpy'
    else:
        return 'NA'

    
sales_week_df['classification'] = sales_week_df.apply(lambda x: classification(x), axis=1)
sales_week_df

sales_week_df.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/0_Data/KA_US_ASIN_classification_weekly.csv', index=False)


category_analysis_df = sales_week_df.groupby('classification').agg({ 'asin':'nunique'}).reset_index()
category_analysis_df

Asin count in base data:  5847
Asin count where gmv_units > 0:  4834


,classification,asin
0,Erratic,1335
1,Intermittent,266
2,Lumpy,2074
3,NA,64
4,Smooth,1095


In [3]:
#Load the data

# Define the file paths
file_path_1 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B1 katchon\0_Data\katchon_Amazon_com_weekly_data.csv"
file_path_2 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B1 katchon\0_Data\KA_US_ASIN_classification_weekly.csv"


# Read the CSV files into DataFrames
df_base1 = pd.read_csv(file_path_1)
df_asin_class = pd.read_csv(file_path_2)


In [4]:
# Filtering input data:

df_asin_class1 = df_asin_class[['asin', 'classification']]

df_base = pd.merge(df_base1, df_asin_class1, on ='asin')

df_weekly = df_base

# df_weekly.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B3 MP/2 CA/df_weekly_base.csv', index=False)

# Convert 'Week_Start_date' to datetime format
df_weekly['Week_Start_date'] = pd.to_datetime(df_weekly['Week_Start_date'])

# Replace NaN values in 'asp_week' column with 0
df_weekly['asp_week'] = df_weekly['asp_week'].fillna(0)

# Filter df_weekly to only include rows where 'classification' is 'Smooth' or 'Intermittent'
# df_weekly = df_weekly[df_weekly['classification'].isin(['Smooth', 'Intermittent'])]


In [5]:
df_weekly

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,inventory_available,classification
0,NaN,B01G07XAWQ,2022-03-28,2022-04-03,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,Lumpy
1,NaN,B01G07XAWQ,2022-04-04,2022-04-10,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,Lumpy
2,NaN,B01G07XAWQ,2022-04-11,2022-04-17,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,Lumpy
3,NaN,B01G07XAWQ,2022-04-18,2022-04-24,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,Lumpy
4,NaN,B01G07XAWQ,2022-04-25,2022-05-01,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,Lumpy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530978,NaN,B0D9MB4VSC,2024-07-29,2024-08-04,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,NaN
530979,NaN,B0D9MB4VSC,2024-08-05,2024-08-11,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,NaN
530980,NaN,B0D9MB4VSC,2024-08-12,2024-08-18,0.0,0.0,0,0.00,0.000000,0.000000,0.00,0.00,0.00,0,NaN
530981,NaN,B0D9MB4VSC,2024-08-19,2024-08-25,0.0,0.0,0,0.00,0.000000,85.714286,0.00,0.00,0.00,300,NaN


In [6]:
#creating New DF 
df_bs_temp = df_weekly
# Display the first few rows of the filtered DataFrame
print(df_bs_temp.head())

#creating ASINx VB for mapping purpose
# Remove rows where 'asin' is null
df_asin_vb = df_bs_temp.dropna(subset=['asin'])

# Select distinct 'asin' x 'velocity_bucket' x classification:
df_asin_vb = df_asin_vb[['asin', 'velocity_bucket', 'classification']].drop_duplicates()

# Display the resulting DataFrame
print("Distinct asin x velocity_bucket:")
print(df_asin_vb.head())




   velocity_bucket        asin Week_Start_date Week_End_date  \
0              NaN  B01G07XAWQ      2022-03-28    2022-04-03   
1              NaN  B01G07XAWQ      2022-04-04    2022-04-10   
2              NaN  B01G07XAWQ      2022-04-11    2022-04-17   
3              NaN  B01G07XAWQ      2022-04-18    2022-04-24   
4              NaN  B01G07XAWQ      2022-04-25    2022-05-01   

   organic_gmv_units  inorganic_gmv_units  gmv_units  asp_week  ad_spend  \
0                0.0                  0.0          0       0.0       0.0   
1                0.0                  0.0          0       0.0       0.0   
2                0.0                  0.0          0       0.0       0.0   
3                0.0                  0.0          0       0.0       0.0   
4                0.0                  0.0          0       0.0       0.0   

   inventory_Availability%  gmv  inorganic_gmv  organic_gmv  \
0                      0.0  0.0            0.0          0.0   
1                      0.0  0.0 

In [7]:
# Split the data into training and testing datasets
train_df = df_weekly[df_weekly['Week_Start_date'] <= '2024-04-29']
test_df = df_weekly[(df_weekly['Week_Start_date'] >= '2024-05-06') & (df_weekly['Week_Start_date'] <= '2024-08-26')]


In [71]:
print("train_df:", train_df)


In [72]:
print("test_df",test_df)

In [8]:
# Ensure 'asin' column exists
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)


Columns in train_df: Index(['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
       'organic_gmv_units', 'inorganic_gmv_units', 'gmv_units', 'asp_week',
       'ad_spend', 'inventory_Availability%', 'gmv', 'inorganic_gmv',
       'organic_gmv', 'inventory_available', 'classification'],
      dtype='object')
Columns in test_df: Index(['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
       'organic_gmv_units', 'inorganic_gmv_units', 'gmv_units', 'asp_week',
       'ad_spend', 'inventory_Availability%', 'gmv', 'inorganic_gmv',
       'organic_gmv', 'inventory_available', 'classification'],
      dtype='object')


In [9]:
#version 2:  
# GMV_units forecasting
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    #model = Prophet(changepoint_prior_scale=0.03)
    #model = Prophet()
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
        
    model = Prophet(
  
                yearly_seasonality=True, 
                weekly_seasonality=False, 
                changepoint_range=1.0,
                # changepoint_prior_scale = 0.000001 #This need to be tuned   
               )
     
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
         print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin] < 0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')

# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)



Processing ASIN: B01G07XAWQ


17:32:49 - cmdstanpy - INFO - Chain [1] start processing
17:32:49 - cmdstanpy - INFO - Chain [1] done processing
17:32:49 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B01H30M0DY


17:32:49 - cmdstanpy - INFO - Chain [1] done processing
17:32:49 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B01MPZZJI1


...

Processing ASIN: B0CZHTDMZ5


18:08:47 - cmdstanpy - INFO - Chain [1] start processing
18:08:48 - cmdstanpy - INFO - Chain [1] done processing
18:08:48 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B0CZLJMP27


18:08:48 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0D1G63WKL
Processing ASIN: B0D1GBZ9X4
Processing ASIN: B0D1GGHTRM
Processing ASIN: B0D1GH4Z4H
Processing ASIN: B0D1GH9M2R
Processing ASIN: B0D1GJ7KPN
Processing ASIN: B0D1GJVF65
Processing ASIN: B0D1GJZM8X
Processing ASIN: B0D1GLZZ23
Processing ASIN: B0D1GMXQ3Q
Processing ASIN: B0D1GN2NFT
Processing ASIN: B0D1GS8ZD8
Processing ASIN: B0D1GSCB2N
Processing ASIN: B0D1GZT1DV
Processing ASIN: B0D2RLR5TQ
Processing ASIN: B0D36RFKR2
Processing ASIN: B0D3733JP5
Processing ASIN: B0D3733SKJ
Processing ASIN: B0D374YT6T
Processing ASIN: B0D375RMLD
Processing ASIN: B0D376FXLQ
Processing ASIN: B0D376VL55
Processing ASIN: B0D3774M7W
Processing ASIN: B0D377XQGF
Processing ASIN: B0D37FGQKK
Processing ASIN: B0D37HQCMB
Processing ASIN: B0D37JJRD3
Processing ASIN: B0D37LSYC8
Processing ASIN: B0D37NP5M9
Processing ASIN: B0D37Q32NM
Processing ASIN: B0D37R7654
Processing ASIN: B0D37S1R42
Processing ASIN: B0D37SYTKQ
Processing ASIN: B0D37W1G48
Processing ASIN: B0D39H2HXX
Processing ASIN: B0D

In [ ]:
X_test_fb2

In [10]:
# Storing Results in a DataFrame and Calculating Metrics:

df_weekly1 = df_weekly[['asin', 'Week_Start_date','gmv_units']]

#Forcast results: setting any negative values to zero
df_results['forecasted_value'] = df_results['forecasted_value'].apply(lambda x: max(0, x))

df_results1 =  pd.merge(df_results, df_weekly1, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])

df_results2 = df_asin_vb.merge(df_results1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results2['absolute_percentage_error'] = abs((df_results2['forecasted_value'] - df_results2['gmv_units']) / df_results2['gmv_units'])

# Calculate Absolute Errors
df_results2['absolute_error'] = abs(df_results2['forecasted_value'] - df_results2['gmv_units'])

# Calculate overall WAPE
overall_wape = df_results2['absolute_error'].sum() / df_results2['gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")

# df_results2

# Calculate WAPE for each date and velocity bucket
wape_per_date = df_results2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date)

# Save the forecast results to a CSV file
# wape_per_date.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_GMV_units.csv', index=False)


# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket = df_results2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date x velocity buckets:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
# wape_per_date_bucket.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_Velocity_buckets_GMV_units.csv', index=False)


# Calculate WAPE : week x class
wape_per_date_asin_class = df_results2.groupby(['Week_Start_date', 'classification']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin x class:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
# wape_per_date_asin_class.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_ASIN_CLASS_WAPE_buckets_GMV_units.csv', index=False)


# Calculate WAPE for asin :
wape_per_asin = df_results2.groupby(['asin']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
# wape_per_asin.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B3 MP/2 CA/MP_CA_Weekly_ASIN_WAPE_GMV_units.csv', index=False)



Overall WAPE: 587.87%

WAPE for each forecast date:
   Week_Start_date      WAPE
0       2024-05-06  0.653364
1       2024-05-13  1.462026
2       2024-05-20  3.601601
3       2024-05-27  6.428031
4       2024-06-03  8.508976
5       2024-06-10  9.122337
6       2024-06-17  7.544339
7       2024-06-24  7.094037
8       2024-07-01  8.196552
9       2024-07-08  5.793585
10      2024-07-15  4.976835
11      2024-07-22  9.308882
12      2024-07-29  8.279543
13      2024-08-05  7.692440
14      2024-08-12  7.952282
15      2024-08-19  8.186020
16      2024-08-26  7.293606


C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_14728\1973983585.py:39: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  wape_per_date_bucket = df_results2.groupby(['Week_Start_date', 'velocity_bucket']).apply(


TypeError: DataFrame.reset_index() got an unexpected keyword argument 'name'

In [11]:
# df_weekly
df_results2

,asin,velocity_bucket,classification,Week_Start_date,actual_outlier_treated,forecasted_value,gmv_units,absolute_percentage_error,absolute_error
0,B01G07XAWQ,NaN,Lumpy,2024-05-06,0,11,0,inf,11
1,B01G07XAWQ,NaN,Lumpy,2024-05-13,0,40,0,inf,40
2,B01G07XAWQ,NaN,Lumpy,2024-05-20,1,91,1,90.0,90
3,B01G07XAWQ,NaN,Lumpy,2024-05-27,0,96,0,inf,96
4,B01G07XAWQ,NaN,Lumpy,2024-06-03,0,91,0,inf,91
...,...,...,...,...,...,...,...,...,...
80328,B0D39KDMNN,NaN,Smooth,2024-07-29,1,0,1,1.0,1
80329,B0D39KDMNN,NaN,Smooth,2024-08-05,3,0,3,1.0,3
80330,B0D39KDMNN,NaN,Smooth,2024-08-12,9,0,9,1.0,9
80331,B0D39KDMNN,NaN,Smooth,2024-08-19,5,0,5,1.0,5


In [17]:
# Result Dataframe: GMV units

sf_base = df_weekly[['velocity_bucket','classification','asin','Week_Start_date', 'gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-26')]

sf_base = sf_base.rename(columns={'gmv_units': 'sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','forecasted_sales','abs_per_error_sales', 'absolute_error_sales']]

#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])


# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','classification','asin','Week_Start_date','sales','forecasted_sales','abs_per_error_sales',
                    'absolute_error_sales',]]

# Save the forecast results to a CSV file
sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/1 US/KA_US_Final_Weekly_forecast_sales_v1.csv', index=False)



In [19]:
# #Rolling up data for reporting

file_path_5 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B1 katchon\0_Data\KA_US_ASIN_80_GMV.csv"

# Read the CSV files into DataFrames
df_selected_asins = pd.read_csv(file_path_5)

# Step 1: Define the mapping of forecast dates to their corresponding months
date_to_month_map = {
    '2024-05-06': '2024-05-01', '2024-05-13': '2024-05-01', '2024-05-20': '2024-05-01', '2024-05-27': '2024-05-01',
    '2024-06-03': '2024-06-01', '2024-06-10': '2024-06-01', '2024-06-17': '2024-06-01', '2024-06-24': '2024-06-01',
    '2024-07-01': '2024-07-01', '2024-07-08': '2024-07-01', '2024-07-15': '2024-07-01', '2024-07-22': '2024-07-01',
    '2024-08-05': '2024-08-01', '2024-08-12': '2024-08-01', '2024-08-19': '2024-08-01', '2024-08-26': '2024-08-01'
}

# Step 2: Filter sf_data to include only the selected ASINs
sf_data2 = sf_data[sf_data['asin'].isin(df_selected_asins['ASIN'])]

# Step 3: Select relevant columns
sf_data2 = sf_data2[['asin', 'Week_Start_date', 'forecasted_sales']]

# Convert Week_Start_date to string (in case it's in datetime format)
sf_data2['Week_Start_date'] = pd.to_datetime(sf_data2['Week_Start_date']).dt.strftime('%Y-%m-%d')

# Step 4: Map Week_Start_date to the month based on the dictionary
sf_data2['month'] = sf_data2['Week_Start_date'].map(date_to_month_map)

# Check if any 'month' values are still NaN after the mapping
unmapped_dates = sf_data2[sf_data2['month'].isna()]['Week_Start_date'].unique()

if len(unmapped_dates) > 0:
    print("The following Week_Start_date values could not be mapped:", unmapped_dates)

# Step 5: Group by 'asin' and 'month', summing 'forecasted_sales'
sf_data3 = sf_data2.groupby(['asin', 'month']).agg({'forecasted_sales': 'sum'}).reset_index()

# Step 6: Save the results to a CSV file
sf_data3.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/1 US/KA_US_Final_Weekly_forecast_sales_month_v1.csv', index=False)


The following Week_Start_date values could not be mapped: ['2024-07-29']


In [18]:
sf_data2

,asin,Week_Start_date,forecasted_sales,month
0,B01G07XAWQ,2024-05-06,11,NaN
1,B01G07XAWQ,2024-05-13,40,NaN
2,B01G07XAWQ,2024-05-20,91,NaN
3,B01G07XAWQ,2024-05-27,96,NaN
4,B01G07XAWQ,2024-06-03,91,NaN
...,...,...,...,...
79274,B0CX989PP5,2024-07-29,0,NaN
79275,B0CX989PP5,2024-08-05,0,NaN
79276,B0CX989PP5,2024-08-12,0,NaN
79277,B0CX989PP5,2024-08-19,0,NaN


In [48]:
#version:  # organic_units forecasting
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['organic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    #model = Prophet(changepoint_prior_scale=0.03)
    model = Prophet()
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin] < 0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')

# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['organic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)

# Storing Results in a DataFrame and Calculating Metrics:

df_weekly1 = df_weekly[['asin', 'Week_Start_date','organic_gmv_units']]

df_results1 =  pd.merge(df_results, df_weekly1, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])

df_results2 = df_asin_vb.merge(df_results1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results2['absolute_percentage_error'] = abs((df_results2['forecasted_value'] - df_results2['organic_gmv_units']) / df_results2['organic_gmv_units'])

# Calculate Absolute Errors
df_results2['absolute_error'] = abs(df_results2['forecasted_value'] - df_results2['organic_gmv_units'])


# Result Dataframe: GMV units

sf_base = df_weekly[['velocity_bucket','classification','asin','Week_Start_date', 'organic_gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-26')]

sf_base = sf_base.rename(columns={'organic_gmv_units': 'organic_sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_organic_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','forecasted_organic_sales','abs_per_error_sales', 'absolute_error_sales']]

#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])


# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','classification','asin','Week_Start_date','organic_sales','forecasted_organic_sales','abs_per_error_sales',
                    'absolute_error_sales',]]

# Save the forecast results to a CSV file
sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/1 US/KA_US_Final_Weekly_forecast_organic_sales.csv', index=False)


Processing ASIN: B075J3GCF5


19:45:17 - cmdstanpy - INFO - Chain [1] start processing
19:45:18 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B075J3JKQ5


19:45:18 - cmdstanpy - INFO - Chain [1] start processing
19:45:18 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B075J3LWVB


...

Processing ASIN: B0CZHRYQLG
Processing ASIN: B0CZHSFBBH


19:57:53 - cmdstanpy - INFO - Chain [1] start processing
19:57:54 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CZHT1H7X


19:57:54 - cmdstanpy - INFO - Chain [1] start processing
19:57:55 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0D1GH9M2R
Error fitting model for ASIN B0D1GH9M2R: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D2RLR5TQ
Processing ASIN: B0D36RFKR2
Error fitting model for ASIN B0D36RFKR2: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D3733SKJ
Error fitting model for ASIN B0D3733SKJ: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D374YT6T
Error fitting model for ASIN B0D374YT6T: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D375RMLD
Error fitting model for ASIN B0D375RMLD: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D376VL55
Error fitting model for ASIN B0D376VL55: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D3774M7W
Error fitting model for ASIN B0D3774M7W: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D377XQGF
Error fitting model for ASIN B0D377XQGF: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D37FGQKK
Error fitting model for ASIN B0D37FGQKK: Dataframe has less than 2 non-N

In [49]:
#version:  # inorganic_units forecasting
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['inorganic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    #model = Prophet(changepoint_prior_scale=0.03)
    model = Prophet()
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin] < 0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')

# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['inorganic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)

# Storing Results in a DataFrame and Calculating Metrics:

df_weekly1 = df_weekly[['asin', 'Week_Start_date','inorganic_gmv_units']]

df_results1 =  pd.merge(df_results, df_weekly1, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])

df_results2 = df_asin_vb.merge(df_results1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results2['absolute_percentage_error'] = abs((df_results2['forecasted_value'] - df_results2['inorganic_gmv_units']) / df_results2['inorganic_gmv_units'])

# Calculate Absolute Errors
df_results2['absolute_error'] = abs(df_results2['forecasted_value'] - df_results2['inorganic_gmv_units'])


# Result Dataframe: GMV units

sf_base = df_weekly[['velocity_bucket','classification','asin','Week_Start_date', 'inorganic_gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-26')]

sf_base = sf_base.rename(columns={'inorganic_gmv_units': 'inorganic_sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_inorganic_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','forecasted_inorganic_sales','abs_per_error_sales', 'absolute_error_sales']]

#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])


# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','classification','asin','Week_Start_date','inorganic_sales','forecasted_inorganic_sales','abs_per_error_sales',
                    'absolute_error_sales',]]

# Save the forecast results to a CSV file
sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/B1 katchon/1 US/KA_US_Final_Weekly_forecast_inorganic_sales.csv', index=False)


Processing ASIN: B075J3GCF5
Processing ASIN: B075J3JKQ5
Processing ASIN: B075J3LWVB
Processing ASIN: B078QCSGSX


20:01:28 - cmdstanpy - INFO - Chain [1] start processing
20:01:28 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B079DCKCGS
Processing ASIN: B079J5GYL9
Processing ASIN: B07BJDN2RR


20:01:30 - cmdstanpy - INFO - Chain [1] start processing
20:01:30 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B07C89JTVW
Processing ASIN: B07CG6WRBM


...

Processing ASIN: B0CZHRYQLG
Processing ASIN: B0CZHSFBBH


20:13:29 - cmdstanpy - INFO - Chain [1] start processing
20:13:30 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CZHT1H7X


20:13:31 - cmdstanpy - INFO - Chain [1] start processing
20:13:34 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0D1GH9M2R
Error fitting model for ASIN B0D1GH9M2R: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D2RLR5TQ
Processing ASIN: B0D36RFKR2
Error fitting model for ASIN B0D36RFKR2: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D3733SKJ
Error fitting model for ASIN B0D3733SKJ: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D374YT6T
Error fitting model for ASIN B0D374YT6T: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D375RMLD
Error fitting model for ASIN B0D375RMLD: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D376VL55
Error fitting model for ASIN B0D376VL55: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D3774M7W
Error fitting model for ASIN B0D3774M7W: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D377XQGF
Error fitting model for ASIN B0D377XQGF: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D37FGQKK
Error fitting model for ASIN B0D37FGQKK: Dataframe has less than 2 non-N

In [171]:
#WAPE and WAPE buckets

file_path_3 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\B4 BH\1 US\0_Data\BH_US_WAPE_Buckets_Forecast_input_adj.csv"
df_data_wape = pd.read_csv(file_path_3)

df_data_wape['absolute_error_AP'] = abs(df_data_wape['Actual'] - df_data_wape['Planned'])
df_data_wape['absolute_error_AF'] = abs(df_data_wape['Actual'] - df_data_wape['Forecasted_sales'])

df_data_wape_may = df_data_wape[df_data_wape['Month'] == '01-05-2024']
df_data_wape_jun = df_data_wape[df_data_wape['Month'] =='01-06-2024']
df_data_wape_july = df_data_wape[df_data_wape['Month'] == '01-07-2024']
df_data_wape_aug = df_data_wape[df_data_wape['Month'] == '01-08-2024']



In [172]:
# df_data_wape_may
# df_data_wape_jun
# df_data_wape_july
df_data_wape_aug

,ASIN,Month,Actual,Planned,Forecasted_sales,absolute_error_AP,absolute_error_AF
15,B08JVLCBMQ,01-08-2024,53,2.0,0.0,51.0,53.0
16,B08JVM6F6L,01-08-2024,1,0.0,0.0,1.0,1.0
17,B08JVMLXK3,01-08-2024,0,0.0,0.0,0.0,0.0
18,B0CKHTB5VR,01-08-2024,194,NaN,0.0,NaN,194.0
19,B0CWHB5K22,01-08-2024,143,NaN,0.0,NaN,143.0
...,...,...,...,...,...,...,...
383,B0D2RMYYTP,01-08-2024,330,NaN,NaN,NaN,NaN
384,B0D2RP6241,01-08-2024,873,NaN,NaN,NaN,NaN
385,B0D2RSCZBD,01-08-2024,4684,NaN,NaN,NaN,NaN
386,B0D2RSSZST,01-08-2024,52,NaN,NaN,NaN,NaN


In [173]:
# Calculate WAPE for asin : May
wape_per_asin_AP_may = df_data_wape_may.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

wape_per_asin_AF_may = df_data_wape_may.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Calculate WAPE for asin : June
wape_per_asin_AP_jun = df_data_wape_jun.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

wape_per_asin_AF_jun = df_data_wape_jun.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')


# Calculate WAPE for asin : July
wape_per_asin_AP_July = df_data_wape_july.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

wape_per_asin_AF_July = df_data_wape_july.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')


# Calculate WAPE for asin : Aug
wape_per_asin_AP_aug = df_data_wape_aug.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

wape_per_asin_AF_aug = df_data_wape_aug.groupby(['ASIN']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1555113617.py:3: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1555113617.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1555113617.py:7: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1555113617.py:12: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1555113617.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Po

In [174]:

# Dictionary of your AP and AF DataFrames with corresponding month values
wape_AP_dfs = {
    'AP_may': (wape_per_asin_AP_may, '01-05-2024'),
    'AP_jun': (wape_per_asin_AP_jun, '01-06-2024'),
    'AP_july': (wape_per_asin_AP_July, '01-07-2024'),
    'AP_aug': (wape_per_asin_AP_aug, '01-08-2024')
}

wape_AF_dfs = {
    'AF_may': (wape_per_asin_AF_may, '01-05-2024'),
    'AF_jun': (wape_per_asin_AF_jun, '01-06-2024'),
    'AF_july': (wape_per_asin_AF_July, '01-07-2024'),
    'AF_aug': (wape_per_asin_AF_aug, '01-08-2024')
}

# Initialize empty lists to store AP and AF DataFrames
combined_AP_results = []
combined_AF_results = []

# Function to categorize WAPE into buckets and add the month column
def categorize_and_add_month(df, month):
    # Assuming df has columns: ['asin', 'WAPE']
    
    conditions = [
        (df['WAPE'] <= 0.10),
        (df['WAPE'] > 0.10) & (df['WAPE'] <= 0.20),
        (df['WAPE'] > 0.20) & (df['WAPE'] <= 0.30),
        (df['WAPE'] > 0.30) & (df['WAPE'] <= 0.40),
        (df['WAPE'] > 0.40) & (df['WAPE'] <= 0.50),
        (df['WAPE'] > 0.50) & (df['WAPE'] <= 0.60),
        (df['WAPE'] > 0.60) & (df['WAPE'] <= 0.70),
        (df['WAPE'] > 0.70) & (df['WAPE'] <= 0.80),
        (df['WAPE'] > 0.80) & (df['WAPE'] <= 0.90),
        (df['WAPE'] > 0.90) & (df['WAPE'] <= 1.00),
        (df['WAPE'] > 1.00)
    ]

    choices = [
        "A. 0 - 10%",
        "B. > 10% - 20%",
        "C. > 20% - 30%",
        "D. > 30% - 40%",
        "E. > 40% - 50%",
        "F. > 50% - 60%",
        "G. > 60% - 70%",
        "H. > 70% - 80%",
        "I. > 80% - 90%", 
        "J. > 90% - 100%",
        "K. > 100%"
    ]

    # Create wape_bucket column
    df['wape_bucket'] = np.select(conditions, choices, default='Other')

    # Add the month column
    df['month'] = month

    return df

# Process each AP DataFrame and add month information
for label, (df, month) in wape_AP_dfs.items():
    combined_AP_results.append(categorize_and_add_month(df, month))

# Process each AF DataFrame and add month information
for label, (df, month) in wape_AF_dfs.items():
    combined_AF_results.append(categorize_and_add_month(df, month))

# Concatenate all AP and AF DataFrames into their respective files
final_AP_combined_df = pd.concat(combined_AP_results, ignore_index=True)
final_AF_combined_df = pd.concat(combined_AF_results, ignore_index=True)

# Save the combined results for AP and AF to separate CSV files
output_AP_path = 'C:/Users/Poovesh Kumar M/Downloads/Planning/B4 BH/1 US/WAPE_summary_AP_May_to_Aug.csv'
output_AF_path = 'C:/Users/Poovesh Kumar M/Downloads/Planning/B4 BH/1 US/WAPE_summary_AF_May_to_Aug.csv'

final_AP_combined_df.to_csv(output_AP_path, index=False)
final_AF_combined_df.to_csv(output_AF_path, index=False)

print(f"AP results saved to {output_AP_path}")
print(f"AF results saved to {output_AF_path}")


AP results saved to C:/Users/Poovesh Kumar M/Downloads/Planning/B4 BH/1 US/WAPE_summary_AP_May_to_Aug.csv
AF results saved to C:/Users/Poovesh Kumar M/Downloads/Planning/B4 BH/1 US/WAPE_summary_AF_May_to_Aug.csv



# Dictionary of your DataFrames with their corresponding labels
wape_dfs = {
    'AP_may': wape_per_asin_AP_may,
    'AF_may': wape_per_asin_AF_may,
    'AP_jun': wape_per_asin_AP_jun,
    'AF_jun': wape_per_asin_AF_jun,
    'AP_july': wape_per_asin_AP_July,
    'AF_july': wape_per_asin_AF_July,
    'AP_aug': wape_per_asin_AP_aug,
    'AF_aug': wape_per_asin_AF_aug
}

# Function to categorize WAPE into buckets and save the results
def categorize_and_save(df, label):
    # Assuming df has columns: ['asin', 'WAPE']
    
    conditions = [
        (df['WAPE'] <= 0.10),
        (df['WAPE'] > 0.10) & (df['WAPE'] <= 0.20),
        (df['WAPE'] > 0.20) & (df['WAPE'] <= 0.30),
        (df['WAPE'] > 0.30) & (df['WAPE'] <= 0.40),
        (df['WAPE'] > 0.40) & (df['WAPE'] <= 0.50),
        (df['WAPE'] > 0.50) & (df['WAPE'] <= 0.60),
        (df['WAPE'] > 0.60) & (df['WAPE'] <= 0.70),
        (df['WAPE'] > 0.70) & (df['WAPE'] <= 0.80),
        (df['WAPE'] > 0.80) & (df['WAPE'] <= 0.90),
        (df['WAPE'] > 0.90) & (df['WAPE'] <= 1.00),
        (df['WAPE'] > 1.00)
    ]

    choices = [
        "A. 0 - 10%",
        "B. > 10% - 20%",
        "C. > 20% - 30%",
        "D. > 30% - 40%",
        "E. > 40% - 50%",
        "F. > 50% - 60%",
        "G. > 60% - 70%",
        "H. > 70% - 80%",
        "I. > 80% - 90%", 
        "J. > 90% - 100%",
        "K. > 100%"
    ]

    # Create wape_bucket column
    df['wape_bucket'] = np.select(conditions, choices, default='Other')

    # Summarize the count of ASINs in each WAPE bucket
    bucket_summary = df.groupby('wape_bucket').size().reset_index(name='asin_count')

    # Sort the result for a cleaner view
    bucket_summary = bucket_summary.sort_values(by='wape_bucket')

    # Display the summary
    print(f"Summary for {label}:")
    print(bucket_summary)

    # Save the results to a CSV file
    output_path = f'C:/Users/Poovesh Kumar M/Downloads/Planning/B4 BH/1 US/WAPE_summary_{label}.csv'
    df.to_csv(output_path, index=False)
    print(f"CSV saved to {output_path}")

# Process each DataFrame in wape_dfs
for label, df in wape_dfs.items():
    categorize_and_save(df, label)


# WAPE: planned vs actual vs forecast

file_path_3 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\WAPE_Buckets_Forecast_data_aug.csv"
df_data_wape = pd.read_csv(file_path_3)



# Calculate Absolute Errors
df_data_wape['absolute_error_AP'] = abs(df_data_wape['Actual'] - df_data_wape['planned_qty'])
df_data_wape['absolute_error_AF'] = abs(df_data_wape['Actual'] - df_data_wape['forecasted_sales'])
df_data_wape['absolute_error_PF'] = abs(df_data_wape['planned_qty'] - df_data_wape['forecasted_sales'])

# # Calculate overall WAPE
# overall_wape = df_data_wape2['absolute_error'].sum() / df_data_wape2['gmv_units'].sum()
# print(f"\nOverall WAPE: {overall_wape:.2%}")

# Calculate WAPE for asin :
wape_per_asin_AP = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Calculate WAPE for asin :
wape_per_asin_AF = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Calculate WAPE for asin :
wape_per_asin_PF = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_PF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin:")
print(wape_per_asin_PF)

# Save the forecast results to a CSV file
# wape_per_asin.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_ASIN_WAPE_GMV_units.csv', index=False)


# df_data_wape
# wape_per_asin_AP
wape_per_asin_AF
# wape_per_asin_PF

# Check the result
# print(wape_per_asin_PF[['asin', 'WAPE', 'wape_bucket']])

# Wape_buckets
df = wape_per_asin_AF


# Assuming df has columns: ['asin', 'WAPE']

conditions = [
    (df['WAPE'] <= 0.10),
    (df['WAPE'] > 0.10) & (df['WAPE'] <= 0.20),
    (df['WAPE'] > 0.20) & (df['WAPE'] <= 0.30),
    (df['WAPE'] > 0.30) & (df['WAPE'] <= 0.40),
    (df['WAPE'] > 0.40) & (df['WAPE'] <= 0.50),
    (df['WAPE'] > 0.50) & (df['WAPE'] <= 0.60),
    (df['WAPE'] > 0.60) & (df['WAPE'] <= 0.70),
    (df['WAPE'] > 0.70) & (df['WAPE'] <= 0.80),
    (df['WAPE'] > 0.80) & (df['WAPE'] <= 0.90),
    (df['WAPE'] > 0.90) & (df['WAPE'] <= 1.00),
    (df['WAPE'] > 1.00)
]

choices = [
    "A. 0 - 10%",
    "B. > 10% - 20%",
    "C. > 20% - 30%",
    "D. > 30% - 40%",
    "E. > 40% - 50%",
    "F. > 50% - 60%",
    "G. > 60% - 70%",
    "H. > 70% - 80%",
    "I. > 80% - 90%", 
    "J > 90% - 100%",
    "K.> 100%"
]

df['wape_bucket'] = np.select(conditions, choices, default='Other')


# Summarize the count of ASINs in each WAPE bucket
bucket_summary = df.groupby('wape_bucket').size().reset_index(name='asin_count')

# Sort the result for a cleaner view (optional)
bucket_summary = bucket_summary.sort_values(by='wape_bucket')

# Display the summary
print(bucket_summary)


# Save the forecast results to a CSV file
df.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/WAPE_summary_AF.csv', index=False)
# bucket_summary.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/bucket_summary_PF.csv', index=False)


# Define the Outlier Treatment Function 
# Plotting Before and After Outlier Treatment

# Store original DataFrame before treatment
df_weekly_original = df_weekly.copy()

# Define the Outlier Treatment Function
def replace_outliers_with_previous_week(group, column):
    Q1 = group[column].quantile(0.25)
    Q3 = group[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = (group[column] < lower_bound) | (group[column] > upper_bound)
    
    # Replace outliers with the previous week's value
    group[column] = np.where(outliers, group[column].shift(1), group[column])
    
    # In case the first value is an outlier and has no previous week to reference, we can forward fill
    group[column] = group[column].fillna(method='ffill')
    
    return group

# Apply the Outlier Treatment
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'gmv_units').reset_index(drop=True)
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'organic_gmv_units').reset_index(drop=True)
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'inorganic_gmv_units').reset_index(drop=True)

# Plotting Before and After Outlier Treatment
def plot_outlier_treatment(df_before, df_after, column, asin_list):
    for asin in asin_list:
        df_before_asin = df_before[df_before['asin'] == asin]
        df_after_asin = df_after[df_after['asin'] == asin]
        
        plt.figure(figsize=(14, 6))
        plt.plot(df_before_asin['Week_Start_date'], df_before_asin[column], marker='o', color='red', label=f'{column} Before Treatment')
        plt.plot(df_after_asin['Week_Start_date'], df_after_asin[column], marker='o', color='blue', label=f'{column} After Treatment')
        plt.title(f"{column} Before and After Outlier Treatment for ASIN: {asin}")
        plt.xlabel("Week Start Date")
        plt.ylabel(column)
        plt.legend()
        plt.grid(True)
        plt.show()

# Use the actual list of ASINs you want to plot
asin_to_plot = ['B07JD9FS28', 'B079ZN8SC9', 'B09S26ZF1N', 'B09PZSKJ5N']
plot_outlier_treatment(df_weekly_original, df_weekly, 'gmv_units', asin_to_plot)
plot_outlier_treatment(df_weekly_original, df_weekly, 'organic_gmv_units', asin_to_plot)
plot_outlier_treatment(df_weekly_original, df_weekly, 'inorganic_gmv_units', asin_to_plot)


%%time

# Forecasting organic_gmv_units


import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']  # Adjust this to match your DataFrame column names

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['organic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results_org = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results_org.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'org_actual_outlier_treated': asin_test_data['organic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results_org = pd.concat(df_results_org, ignore_index=True)



# Storing Results in a DataFrame and Calculating Metrics:

df_base2 = df_base[['asin', 'Week_Start_date','organic_gmv_units']]

df_results_org1 =  pd.merge(df_results_org, df_base2, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])


df_results_org2 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_results_org1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results_org2['absolute_percentage_error'] = abs((df_results_org2['forecasted_value'] - df_results_org2['organic_gmv_units']) / df_results_org2['organic_gmv_units'])

# Calculate Absolute Errors
df_results_org2['absolute_error'] = abs(df_results_org2['forecasted_value'] - df_results_org2['organic_gmv_units'])



# Calculate overall WAPE
overall_wape = df_results_org2['absolute_error'].sum() / df_results_org2['organic_gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")


# Calculate WAPE for each date and velocity bucket
wape_per_date_org = df_results_org2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['organic_gmv_units'].sum()
).reset_index(name='WAPE')
# Save the forecast results to a CSV file
wape_per_date_org.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_ORG_units.csv', index=False)

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_org)


# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket_org = df_results_org2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['organic_gmv_units'].sum()
).reset_index(name='WAPE')



# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_bucket_org)

# Save the forecast results to a CSV file
wape_per_date_bucket_org.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_buckets_ORG_units.csv', index=False)


%%time
# Forecasting inorganic_gmv_units

import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']  # Adjust this to match your DataFrame column names

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_train_fb_pred2 = {}
y_test_fb_pred2 = {}


for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['inorganic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results_iorg = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results_iorg.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'org_actual_outlier_treated': asin_test_data['inorganic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results_iorg = pd.concat(df_results_iorg, ignore_index=True)




# Storing Results in a DataFrame and Calculating Metrics:
df_base3 = df_base[['asin', 'Week_Start_date','inorganic_gmv_units']]

df_results_iorg1 =  pd.merge(df_results_iorg, df_base3, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])


df_results_iorg2 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_results_iorg1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results_iorg2['absolute_percentage_error'] = abs((df_results_iorg2['forecasted_value'] - df_results_iorg2['inorganic_gmv_units']) / df_results_iorg2['inorganic_gmv_units'])

# Calculate Absolute Errors
df_results_iorg2['absolute_error'] = abs(df_results_iorg2['forecasted_value'] - df_results_iorg2['inorganic_gmv_units'])


# Calculate overall WAPE
overall_wape = df_results_iorg2['absolute_error'].sum() / df_results_iorg2['inorganic_gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")

# df_results_iorg2
# Calculate WAPE for each date and velocity bucket
wape_per_date_iorg = df_results_iorg2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['inorganic_gmv_units'].sum()
).reset_index(name='WAPE')
# Save the forecast results to a CSV file
wape_per_date_iorg.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_IORG_units.csv', index=False)

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_iorg)

# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket_iorg = df_results_iorg2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['inorganic_gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_bucket_iorg)

# Save the forecast results to a CSV file
wape_per_date_bucket_iorg.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_buckets_IORG_units.csv', index=False)


# Result Dataframe:


sf_base = df_base[['velocity_bucket','asin','Week_Start_date', 'gmv_units', 'organic_gmv_units', 'inorganic_gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-19')]

sf_base = sf_base.rename(columns={'gmv_units': 'sales','organic_gmv_units': 'organic_sales','inorganic_gmv_units': 'inorganic_sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','outlier_treated_sales','forecasted_sales','abs_per_error_sales', 'absolute_error_sales']]

#organic sales
df_results_org3 = df_results_org2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_organic_sales',
    'absolute_error': 'absolute_error_organic_sales',
    'org_actual_outlier_treated': 'outlier_treated_organic_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_organic_sales'
})

df_results_org3 = df_results_org3[['asin', 'forecast_date','outlier_treated_organic_sales','forecasted_organic_sales','abs_per_error_organic_sales', 'absolute_error_organic_sales']]

#inorganic sales
#organic sales
df_results_iorg3 = df_results_iorg2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_inorganic_sales',
    'absolute_error': 'absolute_error_inorganic_sales',
    'org_actual_outlier_treated': 'outlier_treated_inorganic_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_inorganic_sales'
})

df_results_iorg3 = df_results_iorg3[['asin', 'forecast_date','outlier_treated_inorganic_sales','forecasted_inorganic_sales','abs_per_error_inorganic_sales', 'absolute_error_inorganic_sales']]




#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])
sf_data = pd.merge(sf_data, df_results_org3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])
sf_data = pd.merge(sf_data, df_results_iorg3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])




# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','asin','Week_Start_date',
                    'sales','outlier_treated_sales','forecasted_sales','abs_per_error_sales', 'absolute_error_sales',
                    'organic_sales','outlier_treated_organic_sales','forecasted_organic_sales','abs_per_error_organic_sales', 'absolute_error_organic_sales',
                    'inorganic_sales','outlier_treated_inorganic_sales','forecasted_inorganic_sales','abs_per_error_inorganic_sales', 'absolute_error_inorganic_sales']]


# Step 3: Display the Results & write into csv

sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Forecast_data.csv', index=False)


# before and after standardization for the selected ASINs
import pandas as pd
from prophet import Prophet

# Define exogenous variables and the ASINs of interest
exogenous_vars = ['asp_week', 'ad_spend']
selected_asins = ['B07JD9FS28', 'B079ZN8SC9', 'B09S26ZF1N', 'B09PZSKJ5N']

# List to store results
results = []

for asin in selected_asins:
    print(f"Processing ASIN: {asin}")
    
    # Filter the data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    
    if asin_train_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    df_train = pd.DataFrame()
    df_train['ds'] = asin_train_data['Week_Start_date']
    df_train['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    for var in exogenous_vars:
        df_train[var] = asin_train_data[var]
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    model.fit(df_train)
    
    # Access the model's scaling parameters
    scaling_params = model.extra_regressors
    
    # Compare original and standardized values
    for i in range(len(df_train)):
        result = {'asin': asin, 'Week_Start_date': df_train.iloc[i]['ds']}
        for var in exogenous_vars:
            original_value = df_train.iloc[i][var]
            mean = scaling_params[var]['mu']
            std = scaling_params[var]['std']
            standardized_value = (original_value - mean) / std
            
            result[f'before_{var}'] = original_value
            result[f'standardized_{var}'] = standardized_value
        
        results.append(result)

# Create DataFrame from the results list
df_comparison = pd.DataFrame(results)

# Display the resulting DataFrame
print(df_comparison.head())


#version 3: day level x holidays
# GMV_units forecasting

import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

# Generate U.S. holidays DataFrame
cal = calendar()
holidays = cal.holidays(start=train_df['Week_Start_date'].min(), end=test_df['Week_Start_date'].max(), return_name=True)
holiday_df = pd.DataFrame(data=holidays, columns=['holiday'])
holiday_df = holiday_df.reset_index().rename(columns={'index': 'ds'})

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model with holidays
    model = Prophet(changepoint_prior_scale=0.03, holidays=holiday_df)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)
